In [ ]:
#
#    Notebook de cours AMS-X02 - Cours 2 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Explosion thermique
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#    

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

#  Explosion thermique

Nous abordons la théorie de l’explosion thermique étudiée par Semenov et Frank-Kamenetskii dans la première partie du XXème siècle. Nous considérons un réacteur chimique homogène décrit par l’évolution d’une température adimensionnée $\theta$. La variable $\theta$ vaut zéro au temps initial, elle est monotone croissante et, suite à un emballement thermique, elle va brutalement s’élever et atteindre la température adiabatique de flamme adimensionnée $T_r$ en fin de réaction. Sous certaines hypothèses sur la réaction (constante d’avancement sous la forme d’une loi d’Arrhenius, réaction globale d’ordre un par rapport à la fraction massique de combustible, adimensionnement du temps pour que le temps d’allumage se situe autour de
1 - grande énergie d’activation), que nous de détaillerons pas, on arrive à une équation différentielle de la forme suivante :

$$
d_{\tau} \widetilde{\theta} = f(\widetilde{\theta}) = \exp\Bigg(\frac{\widetilde{\theta}}{1 + \widetilde{\theta} \, / \, \beta}\Bigg) \Bigg(1 - \frac{\widetilde{\theta}}{\overline T_r} \Bigg)
$$

où $\beta$ est directement reliée à l’énergie d’activation ($\beta = \frac{E_a}{RT_0}$) où $E_a$ est l’énergie d’activation de la réaction, $R$ la contante des gaz parfaits et $T_0$ la température initiale dimensionnelle).

In [ ]:
class explosion_with_consumption_1eq_model:

    def __init__(self, Tr, beta):
        self.Tr = Tr
        self.beta = beta 

    def fcn(self, t, theta):
        beta = self.beta
        Tr = self.Tr
        theta_dot = np.exp(theta/(1+(theta/beta))) * (1-theta/Tr) 
        return theta_dot
    
    def jac(self, t, theta):
        beta = self.beta
        Tr = self.Tr
        exp = np.exp(theta/(1+(theta/beta)))
        return (exp/(1+theta/beta)**2) * (1-theta/Tr) - exp/Tr

## Non linéarité du terme source 

La constante $\beta$ pilote le caractère plus ou moins non-linéaire de l’équation comme le montrent la cellule suivante.

In [ ]:
Tr = 200.0

theta = np.linspace(0, Tr, 1000)

fig = make_subplots(rows=2, cols=1, subplot_titles=("Non-linearité du terme source", "Derivée du terme source"))

beta = np.arange(10, 100, 10)
for i, beta_i in enumerate(beta):
    emwc1eq = explosion_with_consumption_1eq_model(Tr, beta_i)
    fcn = emwc1eq.fcn
    jac = emwc1eq.jac
    fig.add_trace(go.Scatter(visible=False, x=theta, y=fcn(0, theta), name='f'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=theta, y=jac(0, theta), name='df/dtheta'), row=2, col=1)
    
fig.data[0].visible = True
fig.data[1].visible = True

# create slider
steps = []
for i, beta_i in enumerate(beta):
    step = dict(method="update", label = f"{beta_i}", 
                args=[{"visible": [(el==2*i) or (el==2*i+1) for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'beta = '}, steps=steps)]

fig.update_layout(sliders=sliders, height=800)
fig.update_xaxes(title_text="theta", row=1, col=1)
fig.update_yaxes(title_text="f(theta)", row=1, col=1)
fig.update_xaxes(title_text="theta", row=2, col=1)
fig.update_yaxes(title_text="df/dtheta", row=2, col=1)
fig['layout']['sliders'][0]['pad']=dict(t= 50)
fig.show()

## Solution

In [ ]:
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.atleast_1d(fcn(tn, yn))

    return ode_result(y, t)

def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*np.array(fcn(tip1, uip1))

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*np.atleast_1d(fcn(tn, yn))
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        sol = root(g, y0, (yn, tn+dt))
        y[:,it+1] = sol.x

    return ode_result(y, t)

In [ ]:
Tr = 200.0
beta = 10. # 10 très illustratif avec nt = 501 / 401 / 301 / 201 / 171

emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
fcn = emwc1eq.fcn
jac = emwc1eq.jac

tini = 0.0
tend = 5.0
nt = np.array([171, 201, 401, 501])

yini = (0.,)

fig =  make_subplots(rows=3, cols=1, subplot_titles=("Solution", "Valeur propre", "Valeur propre . dt"))
color = fig.layout.template.layout.colorway

for i, nt_i in enumerate(nt):
    
    dt = (tend-tini) / (nt_i-1)

    emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
    fcn = emwc1eq.fcn
    jac = emwc1eq.jac

    sol_ref = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)
    sol_fe = forward_euler(tini, tend, nt_i, yini, fcn)
    sol_be = backward_euler(tini, tend, nt_i, yini, fcn)

    fig.add_trace(go.Scatter(visible=False, x=sol_ref.t, y=sol_ref.y[0],  line_color=color[0], legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='sol. ref.'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=sol_fe.y[0], line_color=color[1], legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='Euler explicite'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_be.t, y=sol_be.y[0], line_color=color[2], legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='Euler implicite'), row=1, col=1)

    eig_vals_ref = np.zeros(sol_ref.t.size)
    for it in range(sol_ref.t.size):
        eig_vals_ref[it] = jac(0, sol_ref.y[:,it])
    eig_vals_fe = np.zeros(sol_fe.t.size)
    for it in range(sol_be.t.size):
        eig_vals_fe[it] = jac(0, sol_fe.y[:,it])
    eig_vals_be = np.zeros(sol_be.t.size)
    for it in range(sol_be.t.size):
        eig_vals_be[it] = jac(0, sol_be.y[:,it])

    fig.add_trace(go.Scatter(visible=False, x=sol_ref.t, y=eig_vals_ref, line_color=color[0], legendgroup='eig', 
                         legendgrouptitle_text="Valeur propre", name='sol. ref.'), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=eig_vals_fe, line_color=color[1], legendgroup='eig', 
                         legendgrouptitle_text="Valeur propre", name='Euler explicite'), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_be.t, y=eig_vals_be, line_color=color[2], legendgroup='eig', 
                         legendgrouptitle_text="Valeur propre", name='Euler implicite'), row=2, col=1)

    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=eig_vals_fe*dt, line_color=color[1], legendgroup='eigdt', 
                         legendgrouptitle_text="Valeur propre . dt", name='Euler explicite'), row=3, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_be.t, y=eig_vals_be*dt, line_color=color[2], legendgroup='eigdt', 
                         legendgrouptitle_text="Valeur propre . dt",  name='Euler implicite'), row=3, col=1)

for i in range(3*8, 4*8): fig.data[i].visible = True

# create slider
steps = []
for i, nt_i in enumerate(nt):
    step = dict(method="update", label = f"{nt_i}", 
                args=[{"visible": [(el==8*i) or (el==8*i+1) or (el==8*i+2) or (el==8*i+3) or (el==8*i+4) or (el==8*i+5)
                                   or (el==8*i+6) or (el==8*i+7)  for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(active=3, currentvalue={'prefix': 'nt = '}, steps=steps)]

fig.update_layout(sliders=sliders, height=1000, legend=dict(tracegroupgap=230))
fig.show()